In [ ]:
//Import para Jupyter-notebooks 
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import $ivy.`sh.almond::almond-spark:0.6.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.Column
import org.apache.spark.sql.functions.udf

import java.time.LocalDate
import java.time.format.DateTimeFormatter

val spark = {
  NotebookSparkSession.builder()
    .master("local[1]")
    .getOrCreate()
}

In [ ]:
import spark.implicits._
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this) //jupyter
case class S4ner(name: String,
                 lastName: String,
                 bornOn: String,
                 gender: String,
                 city: String)

In [ ]:
def toGroup: String => String = { bornDate =>
    val formatter = DateTimeFormatter.ofPattern("M/d/yyyy")
    val date = LocalDate.parse(bornDate, formatter)
    date.getYear match {
      case x if x < 1984              => "A"
      case x if x >= 1984 && x < 1990 => "B"
      case _                          => "C"
    }
}

In [ ]:
val people: Dataset[S4ner] = spark.read
      .format("csv")
      .option("sep", ";")
      .option("header", "true")
      .load("all.csv")
      .as[S4ner]

    val city1Girls = people
      .filter(e => e.city.equals("CITY1") && e.gender.equals("F"))
      .map(e => (e.lastName, e.name, toGroup(e.bornOn)))

    val total = city1Girls.count()
    val aGroup = city1Girls.filter(_._3.equals("A")).count()
    val bGroup = city1Girls.filter(_._3.equals("B")).count()
    val cGroup = city1Girls.filter(_._3.equals("C")).count()

    println(s"CITY1: $total")
    println(s"Group A: $aGroup, Group B: $bGroup, Group C: $cGroup")

    city1Girls
      .orderBy("_1")
      .show(20)